# 01. Create COG

Implementation moved to [ns_cogger](./ns_cogger.py). See that file for details.


In [ ]:
from ns_configs import RGB_ASSET_CONFIG, MS_ASSET_CONFIG
from ns_cogger import create_cog
from pathlib import Path

force = False
limit = None
input_dir = Path("/Volumes/Sammy/terra-luma/stac-input")

## RGB COGs

In [10]:
# Get All RGB Tif Files
rgb_tif_files = list(input_dir.rglob("*rgb.tif", case_sensitive=False))

# Limit total (does nothing if limit is None)
rgb_tif_files = rgb_tif_files[0:limit]
print(f"Found {len(rgb_tif_files)} RGB GeoTiff files.")

# Create COG for each
for i, input in enumerate(rgb_tif_files):
    print(f"Processing {input.name} ({i + 1} / {len(rgb_tif_files)}) ...")
    try:
        create_cog(input, input.with_suffix('.cog.tif'), RGB_ASSET_CONFIG, force=force)
    except Exception as e:
        print(f"Error processing: {input.name}")
        print(e)

Found 1 RGB GeoTiff files.
Processing 20241207_SANSSTP002_m3m_50mAGL_ortho_rgb.tif (1 / 1) ...
Successfully converted 20241207_SANSSTP002_m3m_50mAGL_ortho_rgb.tif to 20241207_SANSSTP002_m3m_50mAGL_ortho_rgb.cog.tif.


## MS COGs

In [12]:
# Get All RGB Tif Files
ms_tif_files = list(input_dir.rglob("*ms.tif", case_sensitive=False))

# Limit total (does nothing if limit is None)
ms_tif_files = ms_tif_files[0:limit]
print(f"Found {len(ms_tif_files)} MS GeoTiff files.")

# Create COG for each
for i, input in enumerate(ms_tif_files):
    print(f"Processing {input.name} ({i + 1} / {len(ms_tif_files)}) ...")
    try:
        create_cog(input, input.with_suffix('.cog.tif'), MS_ASSET_CONFIG, force=force)
    except Exception as e:
        print(f"Error processing: {input.name}")
        print(e)

Found 1 MS GeoTiff files.
Processing 20241207_SANSSTP002_m3m_50mAGL_ortho_ms.tif (1 / 1) ...
Successfully converted 20241207_SANSSTP002_m3m_50mAGL_ortho_ms.tif to 20241207_SANSSTP002_m3m_50mAGL_ortho_ms.cog.tif.
